In [1]:
import json

ihops = [json.loads(line) for line in open('ihop_list.jl')]
for ihop in ihops:
    ihop.update({'location': (float(ihop['latitude']), float(ihop['longitude']))})

In [2]:
from ipyleaflet import Map, Marker, MarkerCluster
from IPython.core.display import display

ihop_map = Map(center=(37.8, -96), zoom=4)

marker_cluster = MarkerCluster(
    markers=list(map(lambda ihop: Marker(
        location=ihop['location'],
        title=ihop['address'],
        draggable=False,
    ), ihops))
)
ihop_map.add_layer(marker_cluster)

display(ihop_map)

In [3]:
from ipyleaflet import Map, CircleMarker
from IPython.core.display import display

ihop_map = Map(center=(37.8, -96), zoom=4)

for ihop in ihops:
    ihop_map.add_layer(CircleMarker(
        location=ihop['location'],
        title=ihop['address'],
        draggable=False,
        radius=1,
    ))

display(ihop_map)

Map(center=[37.8, -96], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [4]:
from geopy.distance import geodesic
import multiprocess

NUM_LOCATIONS = 30


def closest_ihops(start_ihop, ihops):
    return sorted(ihops, key=lambda other_ihop: geodesic(start_ihop['location'], other_ihop['location']))


def ihop_radius(start_ihop):
    sorted_ihops = closest_ihops(start_ihop, ihops)
    return geodesic(start_ihop['location'], sorted_ihops[NUM_LOCATIONS]['location']).miles


pool = multiprocess.Pool()
radiuses = pool.map_async(ihop_radius, ihops).get()
ihops_with_distances = list(zip(ihops, radiuses))
ranked_ihops = sorted(ihops_with_distances, key=lambda x: x[1])